# Подготовка данных 
Очищаем и Делим на 80/10/10. К сожалению придется убрать полученные выборки, из-за большого объема. 

In [ ]:
import sys
sys.path.append('./src') # Говорим Python: "загляни в папку src

from data_utils import prepare_data

# Запускаем обработку
# Если твой файл называется по-другому, просто замени имя здесь
prepare_data('data/raw_dataset.txt')

Читаем сырые данные: data/raw_dataset.txt
Очистка текстов...
Очищенный датасет сохранен в data\dataset_processed.csv
Разбиение на выборки...
Все файлы (train/val/test) успешно созданы в папке data/


In [3]:
from transformers import AutoTokenizer

# Используем токенизатор от GPT-2, он отлично работает с текстом
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# У GPT-2 нет стандартного токена отступа, назначим его сами
tokenizer.pad_token = tokenizer.eos_token

c:\Users\nkote\sprint1\text-autocomplete\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Обучение
Ниже закомментирована ячейка для обучения на GPU из-за проблем с ВМ. Обучаем быстро для части данных на CPU.

In [ ]:
'''
import sys
sys.path.append('src')
from next_token_dataset import NextTokenDataset
from torch.utils.data import DataLoader

# Создаем объекты Dataset
train_ds = NextTokenDataset('data/train.csv', tokenizer, max_length=64)
val_ds = NextTokenDataset('data/val.csv', tokenizer, max_length=64)

# Создаем DataLoader (те самые "порции" данных для обучения)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

print(f"Данные готовы! Количество батчей в обучении: {len(train_loader)}")

'''

'\n\n\n'

In [6]:
import pandas as pd
import sys
sys.path.append('./src') # Говорим Python: "загляни в папку src
from next_token_dataset import NextTokenDataset
from torch.utils.data import DataLoader

# Берем 10к строк - это хороший баланс между качеством и скоростью CPU
train_df_10k = pd.read_csv('data/train.csv').sample(n=10000, random_state=42)
val_df_small = pd.read_csv('data/val.csv').sample(n=1000, random_state=42)

train_df_10k.to_csv('data/train_10k.csv', index=False)
val_df_small.to_csv('data/val_10k.csv', index=False)

train_ds = NextTokenDataset('data/train_10k.csv', tokenizer, max_length=64)
val_ds = NextTokenDataset('data/val_10k.csv', tokenizer, max_length=64)

# Batch size 64 на CPU обычно работает быстрее всего за счет векторизации
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

In [10]:
'''
# Берем одну порцию данных
x, y = next(iter(train_loader))

print(f"Формат входа (Batch, Seq_len): {x.shape}") 
print(f"Пример первого токена во входе: {x[0][0]}")
print(f"Пример первого токена в ответе: {y[0][0]}") 
# y[0][0] должен быть равен x[0][1], так как это "следующий" токен
'''

'\n# Берем одну порцию данных\nx, y = next(iter(train_loader))\n\nprint(f"Формат входа (Batch, Seq_len): {x.shape}") \nprint(f"Пример первого токена во входе: {x[0][0]}")\nprint(f"Пример первого токена в ответе: {y[0][0]}") \n# y[0][0] должен быть равен x[0][1], так как это "следующий" токен\n'

In [7]:
import torch
from lstm_model import LSTMAutocompleteModel
from lstm_train import train_model
from torch.utils.data import DataLoader

import torch
from lstm_model import LSTMAutocompleteModel
from lstm_train import train_model



model = LSTMAutocompleteModel(
    vocab_size=tokenizer.vocab_size, 
    embedding_dim=128, 
    hidden_dim=256, 
    n_layers=1  # 1 слой на CPU — это закон, иначе не успеем!
)

# Запускаем 5 эпох. Следи за Loss - он должен уверенно падать.
trained_model = train_model(
    model, 
    train_loader, 
    val_loader, 
    tokenizer, 
    epochs=5, 
    device='cpu'
)


import os
os.makedirs('models', exist_ok=True)
torch.save(trained_model.state_dict(), 'models/lstm_10k_final.pth')
print("Готово!")

Epoch 1: 100%|██████████| 157/157 [24:31<00:00,  9.37s/it, loss=7.14]



--- Epoch 1 Results ---
Loss: 7.8803
ROUGE-1: 0.5942 | ROUGE-2: 0.5642
Prompt: 'i am going to' -> Gen: 'i am going to tovisjoy love'
Prompt: 'today is a' -> Gen: 'today is a col out you stressful makes'
Prompt: 'this movie was' -> Gen: 'this movie was you betweenletesgh from'
------------------------------


Epoch 2: 100%|██████████| 157/157 [21:23<00:00,  8.17s/it, loss=7.26]



--- Epoch 2 Results ---
Loss: 6.8585
ROUGE-1: 0.5876 | ROUGE-2: 0.5579
Prompt: 'i am going to' -> Gen: 'i am going to be thunderot and anth'
Prompt: 'today is a' -> Gen: 'today is afe what thing wanna r'
Prompt: 'this movie was' -> Gen: 'this movie was ok new th certain well'
------------------------------


Epoch 3: 100%|██████████| 157/157 [20:12<00:00,  7.72s/it, loss=6.65]



--- Epoch 3 Results ---
Loss: 6.5233
ROUGE-1: 0.5938 | ROUGE-2: 0.5641
Prompt: 'i am going to' -> Gen: 'i am going to fall for myzz one'
Prompt: 'today is a' -> Gen: 'today is a great of aicky in'
Prompt: 'this movie was' -> Gen: 'this movie was new gl diversity and haha'
------------------------------


Epoch 4: 100%|██████████| 157/157 [21:18<00:00,  8.14s/it, loss=6.06]



--- Epoch 4 Results ---
Loss: 6.2429
ROUGE-1: 0.5863 | ROUGE-2: 0.5569
Prompt: 'i am going to' -> Gen: 'i am going to be our straight da one'
Prompt: 'today is a' -> Gen: 'today is a response deserve to love a'
Prompt: 'this movie was' -> Gen: 'this movie was good people today boring she'
------------------------------


Epoch 5: 100%|██████████| 157/157 [21:30<00:00,  8.22s/it, loss=5.62]



--- Epoch 5 Results ---
Loss: 6.0075
ROUGE-1: 0.5946 | ROUGE-2: 0.5653
Prompt: 'i am going to' -> Gen: 'i am going to end something it r sn'
Prompt: 'today is a' -> Gen: 'today is a bugsucquot uber'
Prompt: 'this movie was' -> Gen: 'this movie was serious of the morning night'
------------------------------
Готово!


Ниже приведено обучение модели на GPU. 

In [ ]:
'''
import torch
import os
from lstm_model import LSTMAutocompleteModel
from lstm_train import train_model
from torch.utils.data import DataLoader

# --- ОПТИМИЗИРОВАННЫЕ ПАРАМЕТРЫ (БЕЗОПАСНО ДЛЯ ПАМЯТИ) ---
BATCH_SIZE = 16   # Сильно уменьшили, чтобы не забивать VRAM (память видеокарты)
HIDDEN_DIM = 128  # Вернули к 128 — это значительно снизит потребление памяти при n_layers=2
EMBED_DIM = 64    # Облегчили эмбеддинги
EPOCHS = 10       # Раз у нас всего 500 строк, поставим 10 эпох для "зазубривания"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Используем устройство: {DEVICE}")



# Инициализируем модель (2 слоя при скрытом слое 128 — хороший баланс)
model = LSTMAutocompleteModel(
    vocab_size=tokenizer.vocab_size, 
    embedding_dim=EMBED_DIM, 
    hidden_dim=HIDDEN_DIM, 
    n_layers=2
)
model.to(DEVICE)

# Запускаем обучение
trained_model = train_model(
    model, 
    train_loader, 
    val_loader, 
    tokenizer, 
    epochs=EPOCHS, 
    device=DEVICE
)

# Сохраняем финальные веса
os.makedirs('models', exist_ok=True)
torch.save(trained_model.state_dict(), 'models/lstm_final.pth')
print("Обучение завершено, модель сохранена!")
'''

## Сравнение моделей LSTM и Трансформера на тестовой выборке по метрикам. 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from eval_transformer_pipeline import evaluate_transformer_rouge
from eval_lstm import evaluate_model_rouge
from next_token_dataset import NextTokenDataset

# 1. Загружаем полный тест
test_df = pd.read_csv('data/test.csv')
test_texts = test_df.iloc[:, 0].tolist()

print(f"Запуск оценки на всей тестовой выборке ({len(test_texts)} примеров)...")

# --- ЗАМЕР DISTILGPT2 ---
# На CPU может занять 5-10 минут для полного теста, но это даст самые точные цифры
tr_r1, tr_r2, _ = evaluate_transformer_rouge(
    model_name="distilgpt2", 
    test_texts=test_texts, 
    device="cpu"
)

# --- ЗАМЕР LSTM ---
# Создаем датасет для всего теста
test_ds = NextTokenDataset('data/test.csv', tokenizer, max_length=64)
test_loader = DataLoader(test_ds, batch_size=32)

r1_lstm, r2_lstm = evaluate_model_rouge(trained_model, test_loader, tokenizer, device="cpu")

print("\n" + "="*50)
print("ИТОГОВЫЕ МЕТРИКИ (FULL TEST)")
print("="*50)
print(f"Модель      | ROUGE-1 | ROUGE-2")
print(f"------------|---------|---------")
print(f"LSTM        | {r1_lstm:.4f}  | {r2_lstm:.4f}")
print(f"DistilGPT2  | {tr_r1:.4f}  | {tr_r2:.4f}")
print("="*50)

Запуск оценки на всей тестовой выборке (159681 примеров)...


Device set to use cpu


Оценка Трансформера distilgpt2...


100%|██████████| 100/100 [00:32<00:00,  3.08it/s]



ИТОГОВЫЕ МЕТРИКИ (FULL TEST)
Модель      | ROUGE-1 | ROUGE-2
------------|---------|---------
LSTM        | 0.6138  | 0.5874
DistilGPT2  | 0.6980  | 0.6544


## Сравнение на первых 10 примерах тестовой выборки

In [13]:
from transformers import pipeline

# Инициализируем пайплайн для GPT2 один раз
gpt2_pipe = pipeline("text-generation", model="distilgpt2", device=-1) # -1 это CPU

print("\nСРАВНЕНИЕ ГЕНЕРАЦИИ (ТОП-10 ПРИМЕРОВ):\n")

for i, original_text in enumerate(test_texts[:10]):
    # Берем первые 3-4 слова как "затравку" (prompt)
    words = original_text.split()
    prompt = " ".join(words[:4]) if len(words) > 4 else " ".join(words[:2])
    
    # --- Предсказание LSTM ---
    trained_model.eval()
    input_ids = torch.tensor([tokenizer.encode(prompt)]).to("cpu")
    with torch.no_grad():
        output = trained_model(input_ids)
        # Мы берем [0], так как модель вернула (logits, hidden_states)
        next_token_id = output[0][0, -1, :].argmax().item()
        lstm_completion = tokenizer.decode([next_token_id])
    
    # --- Предсказание DistilGPT2 ---
    gpt2_res = gpt2_pipe(prompt, max_new_tokens=1, num_return_sequences=1, pad_token_id=50256)
    gpt2_completion = gpt2_res[0]['generated_text'].replace(prompt, "").strip()

    print(f"Пример №{i+1}")
    print(f"Начало:  \"{prompt}...\"")
    print(f"Оригинал: \"{original_text}\"")
    print(f"LSTM:     [+{lstm_completion}]")
    print(f"GPT2:     [+{gpt2_completion}]")
    print("-" * 30)

Device set to use cpu



СРАВНЕНИЕ ГЕНЕРАЦИИ (ТОП-10 ПРИМЕРОВ):

Пример №1
Начало:  "me either..."
Оригинал: "me either"
LSTM:     [+ to]
GPT2:     [+on]
------------------------------
Пример №2
Начало:  "why does the best..."
Оригинал: "why does the best chicken in town have to have a line literally out the door for which i am willing to wait at pm on a sunday night"
LSTM:     [+ of]
GPT2:     [+thing]
------------------------------
Пример №3
Начало:  "just woke up wavos..."
Оригинал: "just woke up wavos soon havent been there in ageeees"
LSTM:     [+ and]
GPT2:     [+after]
------------------------------
Пример №4
Начало:  "lovely day..."
Оригинал: "lovely day"
LSTM:     [+ i]
GPT2:     [+one]
------------------------------
Пример №5
Начало:  "work on..."
Оригинал: "work on a sunday"
LSTM:     [+ the]
GPT2:     [+the]
------------------------------
Пример №6
Начало:  "passed the testgetting my..."
Оригинал: "passed the testgetting my motorcycle license"
LSTM:     [+ phone]
GPT2:     [+car]
-----------------

In [14]:

def generate_final_report(r1_lstm, r2_lstm, tr_r1, tr_r2):
    print("="*50)
    print("АНАЛИТИЧЕСКИЙ ОТЧЕТ ПО ИТОГАМ ЭКСПЕРИМЕНТА")
    print("="*50)
    
    # Сравнительная таблица
    df_results = pd.DataFrame({
        'Метрика': ['ROUGE-1 (Слова)', 'ROUGE-2 (Биграммы)'],
        'LSTM (Наш код)': [r1_lstm, r2_lstm],
        'DistilGPT2 (Transformer)': [tr_r1, tr_r2]
    })
    print(df_results.to_string(index=False))
    print("-" * 50)

    # Логика автоматического вывода
    diff_r1 = tr_r1 - r1_lstm
    
    if diff_r1 > 0.05:
        verdict = (
            "ПОБЕДИТЕЛЬ: DistilGPT2\n"
            "ПРИЧИНА: Трансформер показал значительное превосходство в метриках. "
            "Благодаря механизму Self-Attention он лучше улавливает структуру предложений "
            "и генерирует более естественные окончания текстов."
        )
    else:
        verdict = (
            "ПОБЕДИТЕЛЬ: LSTM (с учетом ресурсов)\n"
            "ПРИЧИНА: Разница в качестве незначительна. LSTM эффективнее "
            "адаптировалась к специфике датасета при гораздо меньших затратах "
            "вычислительных ресурсов. Это оптимальный выбор для данной задачи."
        )
    
    print(verdict)
    print("="*50)

# Запуск (используй свои переменные с метриками)
generate_final_report(r1_lstm, r2_lstm, tr_r1, tr_r2)


АНАЛИТИЧЕСКИЙ ОТЧЕТ ПО ИТОГАМ ЭКСПЕРИМЕНТА
           Метрика  LSTM (Наш код)  DistilGPT2 (Transformer)
   ROUGE-1 (Слова)        0.613770                  0.698022
ROUGE-2 (Биграммы)        0.587376                  0.654399
--------------------------------------------------
ПОБЕДИТЕЛЬ: DistilGPT2
ПРИЧИНА: Трансформер показал значительное превосходство в метриках. Благодаря механизму Self-Attention он лучше улавливает структуру предложений и генерирует более естественные окончания текстов.
